In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import json
import geojson
import requests
import base64
import sqlite3 as db
import cgi
sys.path.append('../')

pd.options.display.max_rows=99

In [2]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'
cnx = db.connect('trailheadDirectBirds_sous.db')
cur = cnx.cursor()

Obtain & clean trailhead data

In [4]:
##pull in trailhead stops from trailheadDirectBirds_sous db
trailheadRef = pd.DataFrame()
trailheadRef = pd.read_sql('select * from trailheadRef;',cnx)
trailheadRef.set_index('index',inplace=True)
trailheadRef.convert_dtypes()

,Route,StopName,Address,Latitude,Longitude
index,,,,,
0,IssaquahAlps,EastSunsetWay,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119
1,IssaquahAlps,HighSchool,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801
2,IssaquahAlps,ChiricoTrail_PooPooPoint,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.02173
3,IssaquahAlps,SquakMountain,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997
4,IssaquahAlps,NullMountain,<NA>,<NA>,<NA>
5,IssaquahAlps,MargaretsWay,"190th Ave SE, Issaquah, WA 98027",47.50662,-122.08666
6,MountSi,MountSi,,47.488966,-122.723044
7,MountSi,MountTeneriffe,"Mount Teneriffe Rd, North Bend, WA 98045",47.490104,-122.709182
8,MountSi,LittleSi,"SE Mt Si Rd, North Bend, WA 98045",47.489366,-122.753833


In [5]:
##For the sake of demonstration please assume the data in the 'trailheadRef' comes from a King County Metro service
##and/or is user-provided.

##set types for error checking, other cleaning
trailheadRef['Latitude'] = trailheadRef['Latitude'].fillna(0.00)
trailheadRef['Longitude'] = trailheadRef['Longitude'].fillna(0.00)
trailheadRef['Latitude'] = trailheadRef['Latitude'].astype('float64',errors='raise')
trailheadRef['Longitude'] = trailheadRef['Longitude'].astype('float64',errors='raise')
trailheadRef['CoordinateTest'] = ''
trailheadRef['CoordinateTest'] = trailheadRef['CoordinateTest'].astype('str')
trailheadRef['AddressTest'] = ''
trailheadRef['AddressTest'] = trailheadRef['AddressTest'].astype('str')
trailheadRef['failCode'] = ''
trailheadRef['failCode'] = trailheadRef['failCode'].astype('str')

##todo #10 finish this evaluator engine

##warnings:
trailheadRef.loc[(abs(trailheadRef['Latitude']) < 1) | (abs(trailheadRef['Longitude']) < 1), 'CoordinateTest'] = 'GEO10'
trailheadRef.loc[(trailheadRef['Address'].isna()) | (trailheadRef['Address'].isnull()) | (trailheadRef['Address'] == ''), 'AddressTest'] = 'GEO20'
##failures:
trailheadRef.loc[(trailheadRef['CoordinateTest'] != '') & (trailheadRef['AddressTest'] != ''), 'failCode'] = 'GEO30'
##update the test below when the coordinate fetch engine is complete to reflect failure of fetch
trailheadRef.loc[(trailheadRef['CoordinateTest'] != '') & (trailheadRef['AddressTest'] == ''), 'failCode'] = 'GEO31'

##TODO: #25 fetch coordinates of trailheads using Nominatim address service
##while trailheadRef['GeoTest'] is 'Fail':
    ##do the thing

##todo #9 write publisher for failed geoTest

trailheadRef_clean = pd.DataFrame()
trailheadRef_clean = trailheadRef[trailheadRef['failCode'] == '']
trailheadRef_clean.drop(columns=['CoordinateTest','AddressTest','failCode'],inplace=True,errors='ignore')


C:\Users\lukew\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Trailhead eBird hotspot update

1. Ask the eBird API for the latest list of hotspots for each trailhead
2. Add new eBird hotspots to the table 'Hotspots' in the trailheadDirectBirds_sous database
3. Update hotspot data

In [6]:
##get current hotspots from sous db
sous_trailheadHotspots = pd.read_sql('select * from hotspots',cnx)
sous_trailheadHotspots.set_index('index',inplace=True)
sous_trailheadHotspots.convert_dtypes()
##hard update on critical dtypes
sous_trailheadHotspots['latestObsDt'] = sous_trailheadHotspots['latestObsDt'].astype('datetime64[ns]')
sous_trailheadHotspots['latestUpdate'] = sous_trailheadHotspots['latestUpdate'].astype('datetime64[ns]')

In [7]:
##fetch all hotspots within 4 kilometers of each trailhead
import time
trailheadHotspots = []

for StopName in trailheadRef_clean.itertuples():
    time.sleep(0.3)
    url = '''https://api.ebird.org/v2/ref/hotspot/geo?lat={}&lng={}&dist=4&fmt=json'''.format(StopName.Latitude,StopName.Longitude)
    ebirdapi_auth_header = {'X-eBirdApiToken': ebird_token}
    ebird_request = requests.get(url,headers=ebirdapi_auth_header)
    ebird_response = pd.DataFrame(ebird_request.json())
    if ebird_request.status_code == requests.codes.ok:
        ebird_response['StopName'] = StopName.StopName
        trailheadHotspots.append(ebird_response)
    ebird_request.raise_for_status()

all_trailheadHotspots = pd.concat(trailheadHotspots)

In [5]:
##TODO #24 write script to pass in new hotspots and update existing hotspots --DONE
today = dt.datetime.today()
pd.DataFrame(all_trailheadHotspots)
all_trailheadHotspots['latestObsDt'] = all_trailheadHotspots['latestObsDt'].astype('datetime64[ns]')
all_trailheadHotspots['latestUpdate'] = today.date()

##sort, remove old data
frames = [all_trailheadHotspots,sous_trailheadHotspots]
hotspots = pd.concat(frames,ignore_index=True)
hotspots.sort_values(by=['StopName','locId','latestObsDt','latestUpdate'],ascending=False,inplace=True)
hotspots.drop_duplicates(subset=['StopName','locId'],keep='first',inplace=True)

NameError: name 'all_trailheadHotspots' is not defined

In [22]:
##load to cooking
##this load into cooking triggers more scripts that update the production table 'Hotspots'
hotspots.to_sql(name='Hotspots_cooking',con=cnx,if_exists='append')

Create a polygon for each hotspot


In [29]:
diff = 0.005    ##equivalent to 0.5 miles when applied
hotspotsGeo = pd.DataFrame()
hotspotsGeo = pd.read_sql('SELECT locId,lat,lng FROM Hotspots', con=cnx)
##TODO #34 remove duplicate hotspots before building table of polygons for each hotspot --DONE
##hotspotsGeo.set_index('index',inplace=True)
hotspotsGeo.sort_values(by=['locId'],ascending=True,inplace=True)
hotspotsGeo.drop_duplicates(subset=['locId'],keep='first',inplace=True)
hotspotsGeo.reset_index()
##TODO #33 write a function to incrament the lat & lng values as needed then output the results to a single column in the format lat,lng --DONE

##Define functions to produce a corner of a polygon for each hotspot (NE,SE,SW,NW)
def NW(x,y):
    return x-diff,y+diff
def NE(x,y):
    return x+diff,y+diff
def SE(x,y):
    return x+diff,y-diff
def SW(x,y):
    return x-diff,y-diff

##apply the functions as new columns
##NOTE that appEEARS only accepts coordinates as (longitude,latitude) which is contrary to geoJSON documentation
hotspotsGeo['NW'] = hotspotsGeo.apply(lambda i: NW(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['NE'] = hotspotsGeo.apply(lambda i: NE(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['SE'] = hotspotsGeo.apply(lambda i: SE(i.lng,i.lat), axis = 1)##.astype(str)
hotspotsGeo['SW'] = hotspotsGeo.apply(lambda i: SW(i.lng,i.lat), axis = 1)##.astype(str)
##TODO #35 (script to support) write the results to the database as a new table with a 'latestUpdate' column --DONE
today = dt.datetime.today()
hotspotsGeo['latestUpdate'] = today.date()
hotspotsGeo['latestUpdate'] = hotspotsGeo['latestUpdate'].astype('datetime64[ns]')
##hotspotsGeo.to_sql(name='hotspotsGeo',con=cnx,if_exists='append')

##TODO #37 compile the polygon generated for each hotspot into a coordinate pack formatted for JSON insert --DONE
##no longer neccessary, geoJSON handles packaging
##for locId in hotspotsGeo.itertuples():
    ##hotspotsGeo['polygon'] = '[[{},{},{},{},{}]]'.format(locId.NW,locId.NE,locId.SE,locId.SW,locId.NW).replace('(','[').replace(')',']')

InterfaceError: Error binding parameter 4 - probably unsupported type.

Log in to the appEEARS service to fetch data

In [9]:
##TODO #40 remove the getpass prompt and replace with a credential manager call
earthdata_baseUrl = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'
appEEARS_username = 'lwylie'
appEEARS_password = 'BdiUPBhUa7ma5ds'
import getpass
NASA_username = getpass.getpass(prompt = 'Enter NASA Earthdata Login Username: ')
NASA_password = getpass.getpass(prompt = 'Enter NASA Earthdata Login Password: ')

In [10]:
##obtain an Earthdata token
##TODO #36 the earthdata API is often under maintenance, write a script to abort this process if a new token cannot be obtain
earthdata_loginURL = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/login'
earthdata_loginRequest = requests.post(earthdata_loginURL,auth=(NASA_username,NASA_password))
earthdata_loginResponse = earthdata_loginRequest.json()
print(earthdata_loginResponse)


{'token_type': 'Bearer', 'token': 'fnfJNXOWdPcUloYPa2rIaz4F8IEQsOouanowKmhGMpjvOkB8XVeWxS9SgDkKWF3WdFBijE7e8wrZBkHykX4MXQ', 'expiration': '2022-02-12T22:31:44Z'}


In [11]:
##Transcribe token, builder header
earthdata_token = earthdata_loginResponse['token']
earthdata_head = {'Authorization': 'Bearer {}'.format(earthdata_token)}

In [ ]:
##TODO #43 write a cooked table of locId's with stats to check against list of hotspots. Drive fetch off difference

Architecture:

1. Make all requests in a loop, producing a dictionary of {'locId': 'earthdata_taskID'}
2. For each locId, pull each .csv into a dataframe then load that dataframe into a database table bearing the name that corresponds with the layer and product. Append the locId.
    EXAMPLE: the contents of the .csv file for 'MCD12Q1-006-LC-Prop1-Statistics.csv' go into the table 'MCD12Q1-006-LC-Prop1-Statistics' in the database

In [164]:
##small frame of hotspots for testing
hotspotsGeo_short = hotspotsGeo.head(2)

In [196]:
##TODO #44 resolve error: {'message': "RecursionError('maximum recursion depth exceeded in __instancecheck__',)"} --DONE
##TODO #45 once all hotspots/locId's have Prop1 and Type3 MCD12Q1 data, convert this block to check for LAI and fPAR
earthdataTaskList = []
earthdata_errorLog = []
for locId in hotspotsGeo_short.itertuples():
    ##time.sleep(5.0)
    ##package polygon as a geoJSON
    yy = locId.locId
    glhf = {"type": "FeatureCollection", "features":
        [{
        "type":"Feature",
            "geometry":
                {"type": "Polygon",
                "coordinates":
                    '[[locId.NW,locId.NE,locId.SE,locId.SW,locId.NW]]'
                },
            "properties": {}}]
        }
    glhf_json = geojson.GeoJSON(glhf)
    ##compile JSON task request
    earthdata_task = {
        'task_type': 'area',
        'task_name': yy,
        'params': {
            'dates': 
                [{"endDate": "12-31", 
                "recurring": True, 
                "startDate": "01-01", 
                "yearRange": [2017, 2019]}],
            'layers': 
                [{"layer": "LC_Prop1", "product": "MCD12Q1.006"}, 
                {"layer": "LC_Type3", "product": "MCD12Q1.006"}],
            'output': {
                'format': {
                    'type': 'netcdf4'}, 
                    'projection': 'geographic'},
        'geo':glhf_json}}
    ##submit the task request
    
##TODO #42 if the request returns anything other than a task_id, stop the script and write the returned issue to the error log --DONE
earthdata_taskReq = requests.post('{}task'.format(earthdata_baseUrl),json=earthdata_task,headers=earthdata_head)
ED_taskResp = pd.DataFrame(earthdata_taskReq.json())

##TODO #46 something about this is broken, fix it
if earthdata_taskReq.status_code == 202:
    ##ED_taskResp = pd.DataFrame(earthdata_taskReq.json())
    earthdataTaskList.append(ED_taskResp)
else: 
    ##ED_taskResp = pd.DataFrame(earthdata_taskReq.json())
    ED_taskResp['statusCode'] = earthdata_taskReq.status_code
    ED_taskResp['request'] = json.dumps(earthdata_task)
    ED_taskResp['reqDate'] = dt.datetime.today()
    earthdata_errorLog.append(ED_taskResp)
    

ValueError: If using all scalar values, you must pass an index

In [191]:
##compile either the error list or continue with the tasks
earthdataTaskList

[]

In [194]:
earthdata_errorLog

[                                             message  statusCode  \
 1  RecursionError('maximum recursion depth exceed...         500   
 
                                              request  \
 1  {"task_type": "area", "task_name": "L10129002"...   
 
                      reqDate  
 1 2022-02-11 16:06:48.048480  ]

In [195]:
##compile either the error list or continue with the tasks

EDTaskList = pd.DataFrame()
try:
    EDTaskList = pd.concat(earthdataTaskList)
except ValueError: 
    try:
        earthdata_errorLog = pd.concat(earthdata_errorLog)
        earthdata_errorLog.to_sql('earthdata_errorlog',con=cnx,if_exists='append')
    except ValueError: raise Exception
else: pd.concat(earthdataTaskList)

In [63]:
##submit the task request
##TODO #42 if the request returns anything other than a task_id, stop the script and write the returned issue to the error log
params = {'limit': 15, 'pretty': True}
earthdata_tasks = requests.get('{}task'.format(earthdata_baseUrl),params=params,headers=earthdata_head).json()

TypeError: list indices must be integers or slices, not list

NASA Data Products:
- LAI: LAI is a measure for the total area of leaves per unit ground area and directly related to the amount of light that can be intercepted by plants. It is defined as the one-sided green leaf area per unit ground surface area (LAI = leaf area / ground area, m2 / m2) in broadleaf canopies. There are three methods used to measure LAI for conifers; this project uses projected (or one-sided, in accordance the definition for broadleaf canopies) needle area per unit ground area.
    - In general, a higher LAI value indicates more leaf coverage
- fPAR: Fraction of absorbed photosynthetically active radiation (fPAR) is the fraction of incoming solar radiation in the spectrum of 400–700 nm that is absorbed by vegetation canopy. Data is provided as a percentage.
- Land Cover Type 3: Annual Leaf Area Index (LAI) classification
- FAO-Land Cover Classification System 1 (LCCS1) land cover layer

In [40]:
for task_id in earthdataTaskList.itertuples():
##TODO #39 check status of the appEEARS task request --DONE
import time
earthdata_tasksResp = requests.get('{}task/{}'.format(earthdata_baseUrl,earthdata_taskID),headers=earthdata_head).json()
starttime = time.time()
while earthdata_tasksResp['status'] !='done':
    print(requests.get('{}task/{}'.format(earthdata_baseUrl,earthdata_taskID),headers=earthdata_head).json()['status'])
    time.sleep(20.0 - ((time.time() - starttime) % 20.0))
print(requests.get('{}task/{}'.format(earthdata_baseUrl,earthdata_taskID),headers=earthdata_head).json()['status'])

done


In [53]:
##TODO #38 get the results of the appEEARS task as a bundle
earthdata_bundle = requests.get('{}bundle/{}'.format(earthdata_baseUrl,earthdata_taskID),).json()

In [54]:
##filter for .csv's
earthdata_files = {}
for f in earthdata_bundle['files']:
    if f['file_type'] in 'csv':
        earthdata_files[f['file_id']] = f['file_name']
    else: continue

earthdata_files


{'da025ae0-405e-4f31-b2ac-267b893c343d': 'MCD12Q1-006-QC-lookup.csv',
 '7bafef38-5c43-4c95-b786-a42062a42fd4': 'MCD12Q1-006-QC-Statistics-QA.csv',
 '3dad5532-8fb8-4cd8-ad45-7bb5777d5da8': 'MCD12Q1-006-LC-Prop1-Statistics.csv',
 '4668d16c-b9a3-4fa8-80bd-3ea1d569cf52': 'MCD12Q1-006-LC-Type3-Statistics.csv'}

In [ ]:
for file_id in earthdata_files:
    allresults = pd.DataFrame()

In [32]:
##IT WORKS
##To read results to a dataframe, pass in earthdata_baseUrl + 'bundle/ + taskID/ + file_id/ + file_name
##example: https://lpdaacsvc.cr.usgs.gov/appeears/api/bundle/098dbb7a-0dfc-4f19-8410-a1db4f91170c/5a51d0f8-362a-4ea3-9e8e-ce1bd783aa62/MCD12Q1-006-LC-Prop1-Statistics.csv
qrf = pd.DataFrame()
qrf = pd.read_csv('https://lpdaacsvc.cr.usgs.gov/appeears/api/bundle/098dbb7a-0dfc-4f19-8410-a1db4f91170c/5a51d0f8-362a-4ea3-9e8e-ce1bd783aa62/MCD12Q1-006-LC-Prop1-Statistics.csv')

In [14]:
##close the connection
cnx.close()